In [2]:
!pip install requests

In [3]:
!pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14347 sha256=31e52d58fbf221bcff503242c7d9a9d77bc0fcb8316d8a5006f64b291217f006
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix)
import wikipediaapi
import spacy
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

In [5]:
user_agent = "Wikipedia searchign bot"
wiki_wiki = wikipediaapi.Wikipedia(
    language='en',
    user_agent=user_agent
)

def fetch_articles_from_category(category_name, max_depth=1, current_depth=0, max_articles=20):
    category = wiki_wiki.page("Category:" + category_name)
    articles = []

    if not category.exists():
        print(f"Category '{category_name}' does not exist.")
        return articles

    for c in category.categorymembers.values():
        if len(articles) >= max_articles:
            break
        if c.ns == wikipediaapi.Namespace.MAIN:
            print(f"Fetching article: {c.title} from {category_name}")
            articles.append((c.title, c.text, category_name))
        elif c.ns == wikipediaapi.Namespace.CATEGORY and current_depth < max_depth:
            articles.extend(fetch_articles_from_category(c.title[9:], max_depth, current_depth + 1, max_articles - len(articles)))

    return articles


categories = ["Mathematics", "Biology", "Geography", "History", "Science"]
max_articles_per_category = 100


all_articles = []
for category in categories:
    articles = fetch_articles_from_category(category, max_depth=1, max_articles=max_articles_per_category)
    all_articles.extend(articles)


df = pd.DataFrame(all_articles, columns=["Title", "Content", "Category"])

df.to_csv("category_wikipedia_articles.csv", index=False)

print(f"{len(all_articles)} articles downloaded and saved to 'category_wikipedia_articles.csv'.")

Fetching article: Mathematics from Mathematics
Fetching article: Beijing Institute of Mathematical Sciences and Applications from Mathematics
Fetching article: Language of mathematics from Mathematics
Fetching article: Law (mathematics) from Mathematics
Fetching article: Glossary of areas of mathematics from Fields of mathematics
Fetching article: Glossary of calculus from Fields of mathematics
Fetching article: List of mathematical theories from Fields of mathematics
Fetching article: Mathematics Subject Classification from Fields of mathematics
Fetching article: Pure mathematics from Fields of mathematics
Fetching article: Lists of mathematics topics from Mathematics-related lists
Fetching article: List of mathematical abbreviations from Mathematics-related lists
Fetching article: List of abstract algebra topics from Mathematics-related lists
Fetching article: Index of accounting articles from Mathematics-related lists
Fetching article: Outline of algebra from Mathematics-related lis